# Análise do Catálogo da Netflix


**SME0221 Inferência Estatística - Cibele Russo - ICMC/USP - São Carlos SP**

Alexandre Souza - 12559506

Eduardo Zaffari Monteiro - 12559490

Lucas Ivars Cadima Ciziks - 12559472

## Introdução
Em 2021, a indústria de entretenimento audiovisual provou que conseguiu superar a pandemia do COVID-19 no que tange ao seu faturamento total. Segundo o relatório anual da Motion Picture Association (MPA), no ano passado foram arrecadados US\$ 99,7 bilhões, número que supera os US\$ 98,1 bilhões de 2019, ano anterior à crise sanitária. 

Da arrecadação total, 72% refere-se ao consumo de mídia digital, o equivalente a US\$ 71,9 bilhões, impulsionado pelos serviços de mídia digital (serviços de *streaming* e *TV Everywhere*), que contavam com mais de 1,3 bilhão de assinaturas ativas no último ano, das quais 221,6 milhões eram da Netflix.

Em se tratando da Netflix, que hoje possui 45,2% da fatia de mercado de serviços de *streaming*, aproximadamente 17,8% de suas assinaturas são da América Latina, contra os 33,2% da EMEA, região que corresponde a Europa, Oriente Médio e África. Contudo, ainda segundo o relatório da MPA, o mercado de serviços digitais aumentou 29% nos países latino-americanos em 2021 (quando comparado a 2020), em contraste com os  21% da região comercial EMEA.

Tendo isso em vista, o objetivo deste relatório é tentar provar, por meio da análise exploratória dos dados e teste de hipóteses, que houve maior investimento em produções de origem europeia em detrimento das de origem latino-americana, explicitando a desigualdade cultural entre as duas regiões.

## Desenvolvimento

In [1]:
# Importação de Bibliotecas
import json
import pandas as pd
import numpy as np
from scipy.stats import t, shapiro, boxcox, mannwhitneyu,ttest_ind, f
import plotly.express as px

### Coleta e Limpeza dos Dados

In [2]:
# Carregando Dados referentes ao Catálogo da Netflix
netflix_raw = pd.read_csv('https://raw.githubusercontent.com/Cizika/netflix-analysis/main/netflix_titles.csv', index_col=0)
netflix_raw.head()

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
show_id,,,,,,,,,,,
s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [3]:
# Lista de Países Europeus
european_list = ["Albania","Andorra","Åland","Austria","Belarus","Belgium",
                 "Bosnia and Herzegovina","Bulgaria","Croatia","Cyprus",
                 "Czech Republic","Denmark","Estonia",
                 "Finland","France","Germany","Gibraltar","Greece",
                 "Hungary","Iceland","Ireland", "Italy","Jersey","Latvia",
                 "Liechtenstein","Lithuania","Luxembourg","Macedonia","Malta",
                 "Moldova","Monaco","Montenegro","Netherlands","Norway","Poland",
                 "Portugal","Romania", "Russia","Serbia","Spain","Sweden",
                 "Switzerland","Slovakia","Slovenia","Ukraine","United Kingdom"]

In [4]:
# Lista de Países Latinos
latin_list = ["Costa Rica","Cuba","Haiti","Panama","Saint Bartélemy","El Salvador",
              "Saint Martin","Argentina","Belize","Bolivia","Brazil","Chile",
              "Colombia","Ecuador","Dominican Republic","Guatemala","Honduras",
              "Mexico","Nicaragua","Paraguay","Peru","Puerto Rico","Uruguay",
              "Venezuela"]

In [5]:
# Excluíndo valores nulos e vazios dos dados
netflix_df = netflix_raw[["country","date_added"]].dropna()

# Tratando a data de adição do filme ao catálogo somento pelo seu ano
netflix_df["date_added"] = netflix_df["date_added"].apply(lambda x: int(x.split(", ")[1]))

# Exibindo dados originais
netflix_df

,country,date_added
show_id,,
s1,United States,2021
s2,South Africa,2021
s5,India,2021
s8,"United States, Ghana, Burkina Faso, United Kin...",2021
s9,United Kingdom,2021
...,...,...
s8802,"United Arab Emirates, Jordan",2016
s8803,United States,2019
s8805,United States,2019


### Extraindo Amostra dos Dados

In [6]:
# Coletando amostra dos filmes adicionados em 2021
amostra_raw = netflix_df[netflix_df["date_added"] == 2021].copy()

# Individualizando país por linha (Tratando filmes com mais de um país de origem)
amostra_raw = amostra_raw.assign(country=amostra_raw["country"].str.split(", ")).explode("country")

# Exibindo amostra raw
amostra_raw

,country,date_added
show_id,,
s1,United States,2021
s2,South Africa,2021
s5,India,2021
s8,United States,2021
s8,Ghana,2021
...,...,...
s8321,Sweden,2021
s8321,Norway,2021
s8435,United States,2021


In [7]:
# A amostra será o Número de Filmes por País
amostra = amostra_raw.groupby("country").size().to_frame()

# Excluíndo valores vazios da amostra
amostra.drop("", inplace=True)

# Renomeando Colunas da Amostra
amostra.reset_index(inplace=True)
amostra.rename(columns={"country":"País", 0: "Número de Filmes" }, inplace=True)

# Exibindo amostra
amostra

,País,Número de Filmes
0,Algeria,1
1,Argentina,12
2,Australia,24
3,Austria,1
4,Bangladesh,1
...,...,...
67,"United Kingdom,",1
68,United States,627
69,Uruguay,2
70,Venezuela,1


In [8]:
# Dicotomizando Amostra por Variável Qualitativa Binária "Região"
# As Regiões de análise serão somente América Latina e Europa

def somente_europa_latinos(row):
  if row["País"] in latin_list:
    return "América Latina"
  elif row["País"] in european_list:
    return "Europa"

# Filtrando apenas por países Europeus e Latinos
amostra["Região"] = amostra.apply(somente_europa_latinos, axis=1)
amostra.dropna(inplace=True)

# Exibindo Amostra com Variável Quantitativa (Número de Filmes)
# E Variável Qualitativa Binária (Região)
amostra.set_index(["Região", "País"]).sort_index(level=0)

Número de Filmes
Região         País                            
América Latina Argentina                     12
               Brazil                        11
               Chile                          2
               Colombia                       9
               Mexico                        21
               Uruguay                        2
               Venezuela                      1
Europa         Austria                        1
               Belgium                        9
               Bulgaria                       2
               Czech Republic                 4
               Denmark                        5
               Finland                        2
               France                        60
               Germany                       40
               Greece                         1
               Hungary                        3
               Iceland                        1
               Ireland                        5
               Italy                         16
               Luxembourg                     1
               Netherlands                    5
               Norway                         3
               Poland                         6
               Portugal                       2
               Romania                        5
               Russia                         4
               Spain                         33
               Sweden                         4
               Switzerland                    3
               United Kingdom               119

### Análise Exploratória dos Dados

In [9]:
# Análise Descritiva dos Latinos
amostra[amostra["Região"] == "América Latina"].describe()

,Número de Filmes
count,7.000000
mean,8.285714
std,7.250616
min,1.000000
25%,2.000000
50%,9.000000
75%,11.500000
max,21.000000


In [10]:
# Análise Descritiva dos Europeus
amostra[amostra["Região"] == "Europa"].describe()

,Número de Filmes
count,24.000000
mean,13.916667
std,26.675044
min,1.000000
25%,2.000000
50%,4.000000
75%,6.750000
max,119.000000


In [11]:
# Esboçando Boxplot com dados da Amostra dicotomizada
boxplot = px.box(amostra, x='Região', y= 'Número de Filmes', color="Região")
boxplot.show()

In [12]:
# Plotando Histograma com dados da Amostra dicotomizada
histograma = px.histogram(amostra, x='Número de Filmes', nbins=50, color="Região")
histograma.show()

### Teste de Normalidade

#### Teste de Shapiro-Wilk

In [13]:
# Função para verificar normalidade de uma amostra
def verificar_normalidade(amostra):
  shapiro_stats, p_valor = shapiro(amostra)

  print(f"Valor p encontrado: {p_valor}")

  if p_valor < 0.05:
    print("Portanto, há evidências para recusar a normalidade, segundo o Teste de Shapiro-Wilk")
  else:
    print("Portanto, não há evidências para recusar a normalidade, segundo o Teste de Shapiro-Wilk")

In [14]:
# Verificando normalidade da amostra original
verificar_normalidade(amostra["Número de Filmes"])

Valor p encontrado: 5.8314886253185705e-09
Portanto, há evidências para recusar a normalidade, segundo o Teste de Shapiro-Wilk


#### Normalizando os dados através de Box-Cox

In [15]:
# Aplicando boxcox sobre a amostra
amostra_normalizada, lam = boxcox(amostra["Número de Filmes"])

# Verificando normalidade da nova amostra
verificar_normalidade(amostra_normalizada)

Valor p encontrado: 0.2368444800376892
Portanto, não há evidências para recusar a normalidade, segundo o Teste de Shapiro-Wilk


In [16]:
# Armazenando valores normalizados na amostra
amostra["Número de Filmes Normalizado"] = amostra_normalizada

# Exibindo Amostra preparada para o Teste de Hipóteses
amostra.set_index(["Região", "País"]).sort_index(level=0)

Número de Filmes  Número de Filmes Normalizado
Região         País                                                          
América Latina Argentina                     12                      1.834409
               Brazil                        11                      1.788064
               Chile                          2                      0.634699
               Colombia                       9                      1.677128
               Mexico                        21                      2.108856
               Uruguay                        2                      0.634699
               Venezuela                      1                      0.000000
Europa         Austria                        1                      0.000000
               Belgium                        9                      1.677128
               Bulgaria                       2                      0.634699
               Czech Republic                 4                      1.165454
               Denmark                        5                      1.317088
               Finland                        2                      0.634699
               France                        60                      2.528090
               Germany                       40                      2.379475
               Greece                         1                      0.000000
               Hungary                        3                      0.956628
               Iceland                        1                      0.000000
               Ireland                        5                      1.317088
               Italy                         16                      1.980438
               Luxembourg                     1                      0.000000
               Netherlands                    5                      1.317088
               Norway                         3                      0.956628
               Poland                         6                      1.434662
               Portugal                       2                      0.634699
               Romania                        5                      1.317088
               Russia                         4                      1.165454
               Spain                         33                      2.303338
               Sweden                         4                      1.165454
               Switzerland                    3                      0.956628
               United Kingdom               119                      2.746334

#### Teste de Mann-Whitney-u

In [17]:
mannwhitneyu(amostra[amostra["Região"] == "América Latina"]["Número de Filmes"],
             amostra[amostra["Região"] == "Europa"]["Número de Filmes"], alternative='two-sided')

MannwhitneyuResult(statistic=89.5, pvalue=0.8121561043505766)

### Teste para Comparação de Variâncias

Definindo as seguintes Variáveis Aleatórias para realizar os Teste:

$L: $ Número de filmes produzidos por países latinos no catálogo da Netflix;

$E: $ Número de filmes produzidos por países europeus no catálogo da Netflix.

Assim, considera-se o seguinte teste de hipóteses para comparar as Variâncias:

$\begin{cases}
   H_0: {\sigma^2_L} = {\sigma^2_E}, contra \\
  H_1: {\sigma^2_L} \neq  {\sigma^2_E}
\end{cases}$

In [18]:
# Coletando amostra Latina Normalizada
amostra_latinos = amostra[amostra["Região"] == "América Latina"]

# Coletando amostra Europeia Normalizada
amostra_europeus = amostra[amostra["Região"] == "Europa"]

In [19]:
# Calculando o tamanho da amostra da América Latina
n_latinos = amostra_latinos["Número de Filmes Normalizado"].count()

# Calculando o tamanho da amostra da Europa
n_europeus = amostra_europeus["Número de Filmes Normalizado"].count()

In [20]:
# Calculando Variância Amostral dos Latinos
var_latinos = amostra_latinos["Número de Filmes Normalizado"].var()

print(f"Em 2021, a variância amostral encontrada para os países latinos foi {var_latinos}.")

# Calculando Variância Amostral dos Europeus
var_europeus = amostra_europeus["Número de Filmes Normalizado"].var()

print(f"Em 2021, a variância amostral encontrada para os países europeus foi {var_europeus}.")

Em 2021, a variância amostral encontrada para os países latinos foi 0.6450111318469602.
Em 2021, a variância amostral encontrada para os países europeus foi 0.6351402598422327.


In [21]:
# Fixando alpha (Erro do Tipo 1) como 5%
alpha = 0.05

# Encontrando Região Crítica através da Distribuição de Fisher-Snedecor
tc_1 = f.ppf(alpha, n_latinos - 1, n_europeus - 1)
tc_2 = f.ppf(1 - alpha, n_latinos - 1, n_europeus-1)

print(f"RC = {{t < {np.round(tc_1,2)} ou t > {np.round(tc_2,2)} }}")

RC = {t < 0.26 ou t > 2.53 }


In [22]:
# Calculando estatística F observada na amostra
F_obs = var_latinos / var_europeus

# Verificando se a F observada está dentro da RC
if F_obs < tc_1 or F_obs > tc_2:
    print("Rejeita-se H0, ou seja, há evidências de que as amostras possuem variâncias diferentes.")
else:
    print("Não se rejeita H0, ou seja, há evidências de que as amostras possuem variâncias iguais.")

Não se rejeita H0, ou seja, há evidências de que as amostras possuem variâncias iguais.


### Teste para Comparação de Médias entre Duas Populações Independentes

Como o Teste de Variância acima apontou que as variâncias amostrais são iguais, realizaremos o **Teste para Comparação de Médias entre Duas Populações Independentes com Variâncias Desconhecidas e Iguais**.

Desse modo, considere a **média populacional** de filmes produzidos por país latino como $\mu_L$ e a **média populacional** de filmes produzidos por país europeu como $\mu_E$. Para a comparação das médias, define-se a estatística $\overline{D}$ tal que:

$$\overline{D} = \overline{L} - \overline{E} \sim N\left(\mu_D = \mu_L - \mu_E, \sigma^2_D = \sigma^2 \left(\frac{1}{n_L} + \frac{1}{n_E}\right) \right)$$

Assim, considera-se o seguinte teste de hipóteses:

$\begin{cases}
 H_0: \mu_D = 0, contra \\
 H_1: \mu_D \neq 0
 \end{cases}$

In [23]:
# Calculando a média de filmes por País da América Latina
mean_latinos = amostra_latinos["Número de Filmes Normalizado"].mean()

print(f"Em 2021, cada país latino produziu, em média, {mean_latinos} filmes (Dados Normalizados).")

# Calculando a média de filmes por País da Europa
mean_europa = amostra_europeus["Número de Filmes Normalizado"].mean()

print(f"Em 2021, cada país europeu produziu, em média, {mean_europa} filmes (Dados Normalizados).")

Em 2021, cada país latino produziu, em média, 1.2396935533471332 filmes (Dados Normalizados).
Em 2021, cada país europeu produziu, em média, 1.191173362345337 filmes (Dados Normalizados).


In [24]:
# Graus de Liberdade
graus_liberdade = n_latinos + n_europeus - 2

# Calculando a Variância Amostral Combinada
var_combinada = ((n_latinos - 1) * var_latinos + (n_europeus - 1) * var_europeus) / graus_liberdade

In [25]:
# Fixando alpha (probabilidade do Erro Tipo 1) como 5%
alpha = 0.05

# Calculando os pontos críticos da Região Crítica
tc_1 = t.ppf(alpha/2, graus_liberdade)
tc_2 = t.ppf(1 - alpha/2, graus_liberdade)
                             
print(f"RC = {{t < {np.round(tc_1,2)} ou t > {np.round(tc_2,2)} }}")

RC = {t < -2.05 ou t > 2.05 }


In [26]:
# Calculando estatística T observada
D_barra = mean_latinos - mean_europa
mi_D = 0 # Sob Hipótese nula

T_obs = (D_barra - mi_D) / np.sqrt(var_combinada * (1/n_latinos + 1/n_europeus))

print(f"A Estatística T observada com a amostra foi {T_obs}.")

A Estatística T observada com a amostra foi 0.14150258471622568.


In [27]:
# Verificando se a estatística T observada está na Região Crítica
if (T_obs < tc_1 or T_obs > tc_2):
  print ('Rejeitamos H0, isto é, há evidências de que a média de produção dos dois continentes é diferente')
else: 
  print('Não rejeitamos H0, ou seja, há evidências de que média de produção de filmes dos dois continentes é a mesma.')

Não rejeitamos H0, ou seja, há evidências de que média de produção de filmes dos dois continentes é a mesma.


In [28]:
# Calculando o valor P
t_stats, pvalue = ttest_ind(amostra_latinos["Número de Filmes Normalizado"], amostra_europeus["Número de Filmes Normalizado"])

print(f"O valor p encontrado foi de {pvalue}, o que confirma que não podemos rejeitar H0.")

O valor p encontrado foi de 0.8884513425902842, o que confirma que não podemos rejeitar H0.


## Conclusão

A partir da normalização dos dados através da transformação de Box-Cox, o que foi necessário para realizar o teste paramétrico de comparação de duas médias, e do teste não paramétrico de Mann-Whitney, foi possível chegar à conclusão que há evidências para apontar que a média de filmes latino-americanos e europeus adicionados à Netflix é a mesma. Esse resultado foi obtido por meio da análise da amostra do catálogo do serviço de *streaming* referente ao ano de 2021, praticando-se o conceito de Inferência Estatística. 

É inegável que as plataformas de *streaming* têm cada vez mais ocupado um espaço na indústria cinematográfica, o que pode ser provado pelo aumento em 150% da receita oriunda de mídias digitais de entretenimento entre 2017 e 2021, segundo a MPA. Portanto, faz-se necessário que haja um investimento equivalente nos diversos mercados audiovisuais ao redor do globo, já que os mercados da LATAM e da EMEA tiveram crescimento proporcional comparado ao da América do Norte, que ainda segue como o polo de produção e consumo desse tipo de produto.

No que tange a Netflix, a empresa se comprometeu, em 2022, a investir US\$ 45 milhões em produções europeias para seu catálogo, ao mesmo passo que está programando lançar mais de 20 produções hispânicas focadas no público latino neste ano. Além disso, em 2021, a companhia anunciou que investiria mais de US\$ 300 milhões em títulos mexicanos com foco global e local.

Em adição à isso, outros serviços de assinatura com foco em entretenimento também divulgaram que investiriam mais em produções latino-americanas. A HBO Max, em 2021, anunciou que possui mais de 100 títulos planejados para o mercado latino que serão lançados na plataforma até 2023. Ainda em 2021, a Amazon mostrou interesse no mercado mexicano e sul-americano ao divulgar que investiria US\$ 2,65 milhões em produções dessas regiões para sua plataforma de *streaming*, o Prime Video.

No mais, é importante citar que, em conjunto ao avanço do acesso à internet, impulsionado principalmente pela pandemia, a América Latina tornou-se o segundo o mercado de *streaming* que mais cresce no mundo, atrás apenas da Ásia e do Oriente Médio. Só em 2021, o número de assinaturas em serviços de vídeo aumentou 43% quando comparado a 2020, chegando a 76 milhões. Esse crescimento é evidenciado pela Análise realizada no presente Relatório acerca da amostra referente ao ano 2021.

## Referências Bibliográficas

- STOLL, Julia. **Number of Netflix paying streaming subscribers worldwide as of 1st quarter 2022, by region**. [*S. l.*], 11 maio 2022. Disponível em: https://www.statista.com/statistics/483112/netflix-subscribers/. Acesso em: 4 jul. 2022.
- VOMERO, Renata. **Mercado de entretenimento global cresce em 2021 impulsionado por streaming e gradual recuperação dos cinemas**. [*S. l.*]: Exibidor, 16 mar. 2022. Disponível em: https://www.exibidor.com.br/noticias/mercado/12565-mercado-de-entretenimento-global-cresce-em-2021-impulsionado-por-streaming-e-gradual-recuperacao-dos-cinemas. Acesso em: 4 jul. 2022.
- 2021 THEME Report. **Motion Picture Association**, [*S. l.*], p. 07-26, 14 mar. 2022. Disponível em: https://www.motionpictures.org/wp-content/uploads/2022/03/MPA-2021-THEME-Report-FINAL.pdf. Acesso em: 4 jul. 2022.
- FORRISTAL, Lauren; PEREZ, Sarah. **Netflix shares down more than 20% after losing 200,000 subscribers in first quarter**. [*S. l.*]: TechCrunch, 19 abr. 2022. Disponível em: 
https://techcrunch.com/2022/04/19/netflix-shares-down-more-than-20-after-losing-200000-subscribers-in-first-quarter/. Acesso em: 4 jul. 2022.
- 2021 Annual Report. **Netflix, Inc.**, [*S. l.*], p. 15-21, 19 abr. 2022. Disponível em: https://s22.q4cdn.com/959853165/files/doc_financials/2021/q4/da27d24b-9358-4b5c-a424-6da061d91836.pdf. Acesso em: 4 jul. 2022.
- WELK, Brian. Netflix to Invest \$45 Million in French and European Cinema. **The Wrap**, [*S.l.*], 22 fev. 2022. Disponível em: https://www.thewrap.com/netflix-to-invest-45-million-in-french-and-european-cinema/. Acesso em: 10 jul. 2022.
- NETFLIX to invest \$300 million in more than 50 Mexican original productions. [*S. l.*]: LABS, 27 jan. 2021. Disponível em: https://labsnews.com/en/news/business/netflix-to-invest-us-300-million-in-mexican-original-productions/. Acesso em: 10 jul. 2022.
- NETFLIX to debut over 20 LatAm original productions this year. [*S. l.*]: Telecompaper, 29 abr. 2022. Disponível em: https://www.telecompaper.com/news/netflix-to-debut-over-20-latam-original-productions-this-year--1422711. Acesso em: 10 jul. 2022.
- GOMEZ, Albert. **Millions in new investments for Latin American film production companies**. [*S. l.*]: Al Día, 2 fev. 2022. Disponível em: https://aldianews.com/en/culture/screen/latam-movie-investments. Acesso em: 10 jul. 2022.
- LATIN America surpasses the U.S. and Europe and is already the second fastest-growing streaming market in the world. [*S. l.*]: LABS, 7 dez. 2021. Disponível em: https://labsnews.com/en/articles/business/latin-america-surpasses-the-u-s-and-europe-and-is-already-the-second-fastest-growing-streaming-market-in-the-world/. Acesso em: 10 jul. 2022.
- PINTO, Flávio. **América Latina: HBO Max planeja 100 produções originais até 2023**. [*S.l.*]: Olhar Digital, 29 abr. 2021. Disponível em: https://olhardigital.com.br/2021/04/29/cinema-e-streaming/america-latina-hbo-max-planeja-100-producoes-originais-ate-2023/. Acesso em: 10 jul. 2022.
- NETFLIX Expands its Latin America Investments, Announcing New Original Series, Diablero, Filmed Entirely in Mexico. [*S. l.*]: Netflix, 3 ago. 2017. Disponível em: https://about.netflix.com/en/news/netflix-expands-its-latin-america-investments-announcing-new-original-series-diablero-filmed-entirely-in-mexico. Acesso em: 10 jul. 2022.